In [1]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np

from benchmark import run_benchmark_sample, Trials

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [3]:
def run_benchmark(include_existing_trials=True):
    for num_vars in range(2, 30):
        for complexity in range(1, 30):
            run_benchmark_sample(num_vars, complexity, num_functions=100, num_inputs=100, include_existing_trials=include_existing_trials)



In [7]:
# TODO: still need to update the function generation before running the full benchmark
run_benchmark()


In [2]:
trials = Trials()
await trials.load_results()

In [8]:
# example code to retrieve results from the first trial of 2 variables 2 complexity
trial = trials.get(num_vars=2, complexity=2)[0]
print("Experimental results:", trial.get_counts())
print("Expected:", trial.total_expected_results())
print("Number of matches:", trial.get_counts().get(trial.total_expected_results(), 0))
print("Shot count:", sum(trial.get_counts().values()))

from function_generator import get_python_function
print("Function used:", end="")
print(get_python_function(trial.statement, trial.variables))


Experimental results: {'000': 9920, '100': 34, '010': 28, '001': 18}
Expected: 000
Number of matches: 9920
Shot count: 10000
Function used:
def f(x0, x1):
    return ((x0) ^ (x0)) and (x1)
    


In [ ]:
# Prepare probability data for plotting
num_vars_data = []
complexity_data = []
probability_data = []

for num_vars in range(2, 30):
        for complexity in range(1, 30):
    
            successes = 0 
            shots = 0
            for trial in trials.get(num_vars=num_vars, complexity=complexity):
                # TODO: consider using average hamming distance instead of perfect matches
                successes += trial.get_counts().get(trial.total_expected_results(), 0)
                shots += sum(trial.get_counts().values())
            if shots == 0:
                 print(f"Warning: no shots for {num_vars} variables, complexity {complexity}; skipping")
                 continue
            
            num_vars_data.append(num_vars)
            complexity_data.append(complexity)
            probability_data.append(successes / shots)
    

In [ ]:

threshold = 0.60
colors = ['green' if p > threshold else 'red' for p in probability_data]


plt.figure(figsize=(10, 10))
plt.scatter(num_vars_data, complexity_data, c=colors, edgecolors='black', alpha=0.75)
plt.xticks(np.arange(min(num_vars_data), max(num_vars_data) + 1, 1))
plt.yticks(np.arange(min(complexity_data), max(complexity_data) + 1, 1))

plt.xlabel('Variables Count')
plt.ylabel('Complexity')
plt.title('Tweedledum & IBM System One Boolean Function Benchmark')
plt.show()